# Purpose
Explore the mellbourne housing data to see if we can find features that can predict the house price

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dansbecker/melbourne-housing-snapshot")

print("Path to dataset files:", path)

# 1. Load Data

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 1000)

In [ ]:
import pandas as pd

# Load the dataset into a DataFrame
data = pd.read_csv(f"{path}/melb_data.csv")

# 2. EDA

Notes on Specific Variables <br>
* Rooms: Number of rooms
* Price: Price in dollars
* Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.
* Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.
* SellerG: Real Estate Agent
* Date: Date sold
* Distance: Distance from CBD
* Regionname: General Region (West, North West, North, North east …etc)
* Propertycount: Number of properties that exist in the suburb.
* Bedroom2 : Scraped # of Bedrooms (from different source)
* Bathroom: Number of Bathrooms
* Car: Number of carspots
* Landsize: Land Size
* BuildingArea: Building Size
* CouncilArea: Governing council for the area

In [ ]:
data.describe(include="all")

## 2.1 Set datatypes
Let's set the correct datatypes before we go further

In [ ]:
datatypes = {
    "Suburb": "category",
    "Address": "string",
    "Rooms": "int8",
    "Type": "category",
    "Price": "float32",
    "Method": "category",
    "SellerG": "category",
    "Date": "datetime64[ns]",
    "Distance": "float32",
    "Postcode": "category",
    "Bedroom2": "category",
    "Bathroom": "category",
    "Car": "category",
    "Landsize": "float32",
    "BuildingArea": "float32",
    "YearBuilt": "category",
    "CouncilArea": "category",
    "Lattitude": "float32",
    "Longtitude": "float32",
    "Regionname": "category",
    "Propertycount": "int32",
}
data = data.astype(datatypes)

## 2.2 Remove redundant rows
Let's remove rows that are duplicated, rows with not house price, lots of na's etc

In [ ]:
# Make a copy of the original data
data_interim = data.copy()

# Check for duplicate rows
print(f"Dupllicated rows all: {data_interim.duplicated().sum()}")

# Check for duplicated columns
print(f"Duplicated columns: {data_interim.columns.duplicated().sum()}")

# Check for duplicated rows based on when the house was sold, its address, and price
dups_subset = data_interim.duplicated(subset=["Address", "Date", "Price"]).sum()
print(f"Duplicated rows based on Address, Date, Price: {dups_subset}")

# Let's drop duplicated rows
data_interim = data.drop_duplicates(subset=["Address", "Date", "Price"])
print(f"Data shape: {data_interim.shape}")

In [ ]:
# Check for missing values
print(
    f"Initial Missing Values: {data_interim.isna().sum() / data_interim.shape[0] * 100}"
)

# Drop columns with more than 35% missing values
threshold = 0.35
cols_to_drop = data_interim.columns[
    data_interim.isna().sum() / data_interim.shape[0] > threshold
]
data_interim = data_interim.drop(columns=cols_to_drop)
print(f"Dropped columns: {list(cols_to_drop)}")

# Impute missing values for 'CouncilArea' & 'Car' with mode
data_interim["CouncilArea"] = data_interim["CouncilArea"].fillna(
    data_interim["CouncilArea"].mode()[0]
)
data_interim["Car"] = data_interim["Car"].fillna(data_interim["Car"].mode()[0])
print(f"Post Impute Missing Values: {data_interim.isna().sum().sum()}")


print(f"Data shape after dropping columns: {data_interim.shape}")

## 2.2 Initial data visual
Let's see which numeric features have strong correlation/association with house prices

### 2.2.1 All numeric data

In [ ]:
import seaborn as sns

sns.pairplot(data_interim, diag_kind="hist", hue="Type")

From the scatterplots, appears to be some signal for predicting price in all the numeric features except Landsize.

### 2.2.3 Categorical data
Let's add some features that may provide a signal and re-try to scatter the data

In [ ]:
# Capture road type from the address
road_types = {
    "ALLY": "ALLEY",
    "ALWY": "ALLEYWAY",
    "AMBL": "AMBLE",
    "ANCG": "ANCHORAGE",
    "APP": "APPROACH",
    "ARC": "ARCADE",
    "ART": "ARTERY",
    "AVE": "AVENUE",
    "BASN": "BASIN",
    "BCH": "BEACH",
    "BEND": "BEND",
    "BLK": "BLOCK",
    "BVD": "BOULEVARD",
    "BRCE": "BRACE",
    "BRAE": "BRAE",
    "BRK": "BREAK",
    "BDGE": "BRIDGE",
    "BDWY": "BROADWAY",
    "BROW": "BROW",
    "BYPA": "BYPASS",
    "BYWY": "BYWAY",
    "CAUS": "CAUSEWAY",
    "CTR": "CENTRE",
    "CNWY": "CENTREWAY",
    "CH": "CHASE",
    "CIR": "CIRCLE",
    "CLT": "CIRCLET",
    "CCT": "CIRCUIT",
    "CRCS": "CIRCUS",
    "CL": "CLOSE",
    "CLDE": "COLONNADE",
    "CMMN": "COMMON",
    "CON": "CONCOURSE",
    "CPS": "COPSE",
    "CNR": "CORNER",
    "CSO": "CORSO",
    "CT": "COURT",
    "CTYD": "COURTYARD",
    "COVE": "COVE",
    "CRES": "CRESCENT",
    "CRST": "CREST",
    "CRSS": "CROSS",
    "CRSG": "CROSSING",
    "CRD": "CROSSROAD",
    "COWY": "CROSSWAY",
    "CUWY": "CRUISEWAY",
    "CDS": "CUL-DE-SAC",
    "CTTG": "CUTTING",
    "DALE": "DALE",
    "DELL": "DELL",
    "DEVN": "DEVIATION",
    "DIP": "DIP",
    "DSTR": "DISTRIBUTOR",
    "DR": "DRIVE",
    "DRWY": "DRIVEWAY",
    "EDGE": "EDGE",
    "ELB": "ELBOW",
    "END": "END",
    "ENT": "ENTRANCE",
    "ESP": "ESPLANADE",
    "EST": "ESTATE",
    "EXP": "EXPRESSWAY",
    "EXTN": "EXTENSION",
    "FAWY": "FAIRWAY",
    "FTRK": "FIRE",
    "FITR": "FIRETRAIL",
    "FLAT": "FLAT",
    "FOLW": "FOLLOW",
    "FTWY": "FOOTWAY",
    "FSHR": "FORESHORE",
    "FORM": "FORMATION",
    "FWY": "FREEWAY",
    "FRNT": "FRONT",
    "FRTG": "FRONTAGE",
    "GAP": "GAP",
    "GDN": "GARDEN",
    "GTE": "GATE",
    "GDNS": "GARDENS",
    "GTES": "GATES",
    "GLD": "GLADE",
    "GLEN": "GLEN",
    "GRA": "GRANGE",
    "GRN": "GREEN",
    "GRND": "GROUND",
    "GR": "GROVE",
    "GLY": "GULLY",
    "HTS": "HEIGHTS",
    "HRD": "HIGHROAD",
    "HWY": "HIGHWAY",
    "HILL": "HILL",
    "INTG": "INTERCHANGE",
    "INTN": "INTERSECTION",
    "JNC": "JUNCTION",
    "KEY": "KEY",
    "LDG": "LANDING",
    "LANE": "LANE",
    "LNWY": "LANEWAY",
    "LEES": "LEES",
    "LINE": "LINE",
    "LINK": "LINK",
    "LT": "LITTLE",
    "LKT": "LOOKOUT",
    "LOOP": "LOOP",
    "LWR": "LOWER",
    "MALL": "MALL",
    "MNDR": "MEANDER",
    "MEW": "MEW",
    "MEWS": "MEWS",
    "MWY": "MOTORWAY",
    "MT": "MOUNT",
    "NOOK": "NOOK",
    "OTLK": "OUTLOOK",
    "PDE": "PARADE",
    "PARK": "PARK",
    "PKLD": "PARKLANDS",
    "PKWY": "PARKWAY",
    "PART": "PART",
    "PASS": "PASS",
    "PATH": "PATH",
    "PHWY": "PATHWAY",
    "PIAZ": "PIAZZA",
    "PL": "PLACE",
    "PLAT": "PLATEAU",
    "PLZA": "PLAZA",
    "PKT": "POCKET",
    "PNT": "POINT",
    "PORT": "PORT",
    "PROM": "PROMENADE",
    "QUAD": "QUAD",
    "QDGL": "QUADRANGLE",
    "QDRT": "QUADRANT",
    "QY": "QUAY",
    "QYS": "QUAYS",
    "RMBL": "RAMBLE",
    "RAMP": "RAMP",
    "RNGE": "RANGE",
    "RCH": "REACH",
    "RES": "RESERVE",
    "REST": "REST",
    "RTT": "RETREAT",
    "RIDE": "RIDE",
    "RDGE": "RIDGE",
    "RGWY": "RIDGEWAY",
    "ROWY": "RIGHT",
    "RING": "RING",
    "RISE": "RISE",
    "RVR": "RIVER",
    "RVWY": "RIVERWAY",
    "RVRA": "RIVIERA",
    "RD": "ROAD",
    "RDS": "ROADS",
    "RDSD": "ROADSIDE",
    "RDWY": "ROADWAY",
    "RNDE": "RONDE",
    "RSBL": "ROSEBOWL",
    "RTY": "ROTARY",
    "RND": "ROUND",
    "RTE": "ROUTE",
    "ROW": "ROW",
    "RUE": "RUE",
    "RUN": "RUN",
    "SWY": "SERVICE",
    "SDNG": "SIDING",
    "SLPE": "SLOPE",
    "SND": "SOUND",
    "SPUR": "SPUR",
    "SQ": "SQUARE",
    "STRS": "STAIRS",
    "SHWY": "STATE",
    "STPS": "STEPS",
    "STRA": "STRAND",
    "ST": "STREET",
    "STRP": "STRIP",
    "SBWY": "SUBWAY",
    "TARN": "TARN",
    "TCE": "TERRACE",
    "THOR": "THOROUGHFARE",
    "TLWY": "TOLLWAY",
    "TOP": "TOP",
    "TOR": "TOR",
    "TWRS": "TOWERS",
    "TRK": "TRACK",
    "TRL": "TRAIL",
    "TRLR": "TRAILER",
    "TRI": "TRIANGLE",
    "TKWY": "TRUNKWAY",
    "TURN": "TURN",
    "UPAS": "UNDERPASS",
    "UPR": "UPPER",
    "VALE": "VALE",
    "VDCT": "VIADUCT",
    "VIEW": "VIEW",
    "VLLS": "VILLAS",
    "VSTA": "VISTA",
    "WADE": "WADE",
    "WALK": "WALK",
    "WKWY": "WALKWAY",
    "WAY": "WAY",
    "WHRF": "WHARF",
    "WYND": "WYND",
    "YARD": "YARD",
}


def set_road_type(address: str) -> str:
    """Extract road type from address string."""
    road_type = address.split(" ")[-1]
    if len(road_type) == 1:
        road_type = address.split(" ")[-2] + " " + address.split(" ")[-1]
    if "St" in road_type:
        road_type = "St"
    if "Rd" in road_type:
        road_type = "Rd"
    if "Pl" in road_type:
        road_type = "Pl"
    if "Pde" in road_type:
        road_type = "Pde"
    if "Cr" in road_type:
        road_type = "Cres"
    if "Wy" in road_type:
        road_type = "Way"
    if "La" in road_type:
        road_type = "Lane"
    if "Av" in road_type:
        road_type = "Ave"
    if "The" in address and len(address.split(" ")) == 3:
        road_type = "Other"
    return road_types.get(road_type.upper(), "OTHER")


data_interim.loc[:, "RoadType"] = data_interim["Address"].apply(
    lambda x: set_road_type(str(x))
)
data_interim = data_interim.astype({"RoadType": "category"})

In [ ]:
import matplotlib.pyplot as plt

# 1. Create a 2x2 subplot grid
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))

# 2. Draw each violin plot on its specific axis using the 'ax' parameter
sns.violinplot(data=data_interim, x="Type", y="Price", ax=axes[0, 0])
axes[0, 0].set_title("Price Distribution by Type")

sns.violinplot(data=data_interim, x="Method", y="Price", ax=axes[0, 1])
axes[0, 1].set_title("Price Distribution by Method")

sns.violinplot(data=data_interim, x="Regionname", y="Price", ax=axes[1, 0])
axes[1, 0].set_title("Price Distribution by Regionname")
# Optional: Rotate labels if 'Regionname' has many categories
axes[1, 0].tick_params(axis="x", rotation=45)

sns.violinplot(data=data_interim, x="RoadType", y="Price", ax=axes[1, 1])
axes[1, 1].set_title("Price Distribution by RoadType")

# 3. Adjust the layout to prevent titles/labels from overlapping
plt.tight_layout()

# 4. Display the plot
plt.show()

From the violin plots, appears to be some signal for predicting price in Type which have much tigher range in units and type. Additionally seems to be some signal on the road types which is a positive sign seeing it was engineered

# 3. Regression
Let's attempt regression on the following ML algorithms to select the best algorithm for predicting house prices


## 3.1 Pick Features

In [ ]:
## Remove string or date columns for modeling
cols_to_remove = data_interim.select_dtypes(
    include=["string", "datetime64[ns]"]
).columns
data_interim = data_interim.drop(columns=cols_to_remove)

In [ ]:
categorical_cols = data_interim.select_dtypes(include="category").columns
numeric_cols = data_interim.select_dtypes(include=["number"]).columns
target = ["Price"]
print("Categorical columns:", list(categorical_cols))
print("Numeric columns:", list(numeric_cols))
print(
    "Categorical + Numeric == Original Shape:",
    len(categorical_cols) + len(numeric_cols) == data_interim.shape[1],
)

## 3.2 Transform Data

In [ ]:
## Scale numeric features
from sklearn.preprocessing import StandardScaler

# create scaler
scaler = StandardScaler()

# apply transform
standardized_data = scaler.fit_transform(data_interim[numeric_cols])
standardized_data = pd.DataFrame(
    standardized_data,
    columns=scaler.get_feature_names_out(numeric_cols),
    index=data_interim.index,
)

In [ ]:
## Encode categorical features
from sklearn.preprocessing import OneHotEncoder

# create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
# fit and transform
encoded_data = encoder.fit_transform(data_interim[categorical_cols])
encoded_data = pd.DataFrame(
    encoded_data,
    columns=encoder.get_feature_names_out(categorical_cols),
    index=data_interim.index,
)

In [ ]:
# Combine standardized numeric data and encoded categorical data
data_transformed = pd.concat([standardized_data, encoded_data], axis=1)

## 3.3 Split Data

In [ ]:
features = data_transformed.columns.tolist()
features = [col for col in features if col != "Price"]
target = ["Price"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data_transformed[features], data_transformed[target], test_size=0.2, random_state=42
)

## 3.3 Fit Data
Will fit different models with their default settings to gauge quickly which model offers the best score before picking one and tunning the hyperparamters.

### 3.3.1 Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)
coefficients = pd.Series(lr_model.coef_[0], index=features).sort_values()

print(f"Model Score (Training): {lr_model.score(X_train, y_train)}")
print(f"Model Score (Testing): {lr_model.score(X_test, y_test)}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Create a new dictionary to hold the *grouped* importances
grouped_importance = {}

# 2. Add the numeric features (we just take their absolute value)
for col in numeric_cols:
    if col in coefficients:
        grouped_importance[col] = abs(coefficients[col])

# 3. Group and sum the categorical features
for col in categorical_cols:
    # Find all OHE columns that start with this category name (e.g., "Regionname_...")
    ohe_cols = [c for c in coefficients.index if c.startswith(col + "_")]

    # Sum the absolute values of all their coefficients
    total_cat_importance = coefficients[ohe_cols].abs().sum()

    # Add to our dictionary
    grouped_importance[col] = total_cat_importance

# 4. Convert to a Series, sort, and plot
grouped_coef_series = pd.Series(grouped_importance).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=grouped_coef_series.values, y=grouped_coef_series.index, orient="h")
plt.title("Grouped Feature Importance (Sum of Abs(Coefficients))")
plt.xlabel("Total Importance")
plt.ylabel("Original Feature")
plt.show()

Interesingly the most important predictor of house prices in Melbourne is the real eastate agent. It is suprising as my initial thoughts would have been suburb or distance to city. I suppose it makes sense as certain agents only sell at certain locations

### 3.3.2 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42).fit(X_train, y_train)
print(f"Model Score (Training): {rf_model.score(X_train, y_train)}")
print(f"Model Score (Testing): {rf_model.score(X_test, y_test)}")

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# # 1. Define the model
# rf_model_2 = RandomForestRegressor(random_state=42)

# # 2. Define your wide parameter grid
# param_grid_random = {
#     "n_estimators": [100, 300, 500],
#     "max_features": [0.5, "sqrt", 1.0],  # For regression, '1.0' is the default
#     "max_depth": [10, 20, 30, None],
#     "min_samples_leaf": [1, 2, 4],
#     "min_samples_split": [2, 5, 10],
# }

# # 3. Set up the Random Search
# # n_iter=25 means it will try 25 different random combinations
# # cv=5 is 5-fold cross-validation
# # n_jobs=-1 uses all the computer's cores to run faster
# rf_random_search = RandomizedSearchCV(
#     estimator=rf_model_2,
#     param_distributions=param_grid_random,
#     n_iter=25,
#     cv=5,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1,
# )

# # 4. Run the search
# rf_random_search.fit(X_train, y_train)

In [ ]:
rf_random_search.best_params_

In [ ]:
best_params = {
    "n_estimators": 100,
    "min_samples_split": 5,
    "min_samples_leaf": 2,
    "max_features": 0.5,
    "max_depth": 20,
}

In [ ]:
rf_model_final = RandomForestRegressor(random_state=42, **best_params).fit(
    X_train, y_train
)
print(f"Model Score (Training): {rf_model_final.score(X_train, y_train)}")
print(f"Model Score (Testing): {rf_model_final.score(X_test, y_test)}")

As expected, the tree based regressors are an improvement over the simpler linear regression model. It will be interesting to see how much more boost in performance we can gain from the gradient boosting model

### 3.3.3 Gradient boosting (light gbm)

In [ ]:
from lightgbm import LGBMRegressor

lgr = LGBMRegressor().fit(X_train, y_train)
print(f"Model Score (Training): {lgr.score(X_train, y_train)}")
print(f"Model Score (Testing): {lgr.score(X_test, y_test)}")